# Analysis of Android App Build Reproducibility

This notebook analyzes the reproducibility of Android app builds across different environments and build methods.

In [60]:
import os
import shutil
import pathlib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import hashlib
import itertools
import json
from collections import defaultdict

## Methodology

To test the reproducibility of the Android application, we have followed [the guide in the official repository](https://github.com/signalapp/Signal-Android/tree/main/reproducible-builds). Initially, we have run each step manually, and then wrote [a custom script](https://github.com/TheTechZone/reproducible-tests/blob/main/build_signal.py) to automate the process and prevent errors that could be introduced by humans.
Each build output was obtained using a clean<a name="clean"></a>[<sup>[clean]</sup>](#clean) Docker container. 

All tests very carried again version `v7.25.2` of the Signal application, which the most recent version currently available on Play Store (`region=CH`, `releaseChannel=production`). [According to public info](https://github.com/signalapp/Signal-Android/issues/13754#issuecomment-2450435519), this version should include the recent proguard fixes that ensure build determinism (TODO: link the commits)

The only variability in environment consists of the host operating system that was used (`fedora-40`, `fedora-41`, and `ubuntu-24.04`). Each machine was connected to a diffrent physical Android device for retrieving the play store app.

Example of footnote.


<a name="clean"></a> [^clean](#clean) Before invoking Gradle, the Docker image was built from scratch with no caching to prevent build artifacts from affecting reproducibility.  -- see the `./clean` script
[^clean]: )

## Data Collection

First, let's collect information about all the builds in our outputs directory.

In [2]:
def get_file_hash(filepath):
    """Calculate SHA-256 hash of a file."""
    sha256_hash = hashlib.sha256()
    with open(filepath, "rb") as f:
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()


def collect_build_data(output_dir):
    """Collect build data from the outputs directory."""
    builds_data = []

    def human_size(bytes, units=[" bytes", "KB", "MB", "GB", "TB", "PB", "EB"]):
        """Returns a human readable string representation of bytes"""
        return (
            str(bytes) + units[0]
            if bytes < 1024
            else human_size(bytes >> 10, units[1:])
        )

    for build_dir in Path(output_dir).iterdir():
        if not build_dir.is_dir():
            continue

        # Parse environment info from directory name
        env_info = build_dir.name.split("-")
        os_name = env_info[0]
        build_type = env_info[1]
        execution_count = 1 if len(env_info) == 2 else int(env_info[2])

        # Process APKs
        for apk_dir in build_dir.glob("**/apks*"):
            apk_type = apk_dir.name
            for apk_file in apk_dir.glob("**/*.apk"):
                size = apk_file.stat().st_size
                builds_data.append(
                    {
                        "os": os_name,
                        "build_type": build_type,
                        "exec_count": execution_count,
                        "apk_type": apk_type,
                        "local_apk": apk_type == "apks-i-built",
                        "filename": apk_file.name,
                        "filepath": str(apk_file),
                        "hash": get_file_hash(apk_file),
                        "size": size,
                        "human_size": human_size(size),
                    }
                )

    return pd.DataFrame(builds_data)


# Collect data
output_dir = "outputs"
df = collect_build_data(output_dir)
df.head(6)

,os,build_type,exec_count,apk_type,local_apk,filename,filepath,hash,size,human_size
0,fedora40,scripted,1,apks-from-device,False,base-master.apk,outputs/fedora40-scripted-1/apks-from-device/b...,eb1902b3aa98e15a140e9de574dfa47b78bf8bb85fc556...,85037580,81MB
1,fedora40,scripted,1,apks-from-device,False,base-arm64_v8a.apk,outputs/fedora40-scripted-1/apks-from-device/b...,d296fff8691b3632f85725202045875af7e89dafedc9df...,23569519,22MB
2,fedora40,scripted,1,apks-from-device,False,base-xxhdpi.apk,outputs/fedora40-scripted-1/apks-from-device/b...,c65a2691cde28650f8934d492bff6c241b2e55eee4d222...,1666719,1MB
3,fedora40,scripted,1,apks-i-built,True,base-xxhdpi.apk,outputs/fedora40-scripted-1/apks-i-built/base-...,51119b048c0e03525cbb253ed7c19974738b58804908d3...,1658464,1MB
4,fedora40,scripted,1,apks-i-built,True,base-arm64_v8a.apk,outputs/fedora40-scripted-1/apks-i-built/base-...,a74d526fd13f4fe3bda7e066c575fa0c931d8da7333b8d...,23561264,22MB
5,fedora40,scripted,1,apks-i-built,True,base-master.apk,outputs/fedora40-scripted-1/apks-i-built/base-...,4768602685c6e5ceff078093c87e9aba5f19597b0d76c5...,85021050,81MB


In [3]:
def print_tree_with_hashes(df):
    """Print directory tree with file hashes."""
    # Create a nested dictionary structure
    tree = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))

    # ANSI color codes
    BLUE = "\033[34m"
    PURPLE = "\33[35m"
    BOLD = "\033[1m"
    RESET = "\033[0m"

    # Populate the tree structure
    for _, row in df.iterrows():
        tree[row["os"]][row["build_type"]][row["apk_type"]][row["filename"]] = row[
            "hash"
        ][
            :8
        ]  # First 8 chars of hash

    # Print the tree
    for os in sorted(tree.keys()):
        print(f"{BLUE}{BOLD}{os}/{RESET}")
        for build_type in sorted(tree[os].keys()):
            print(f"├── {BLUE}{BOLD}{build_type}/{RESET}")
            for apk_type in sorted(tree[os][build_type].keys()):
                print(f"│   ├── {BLUE}{BOLD}{apk_type}/{RESET}")
                for filename, hash_prefix in sorted(
                    tree[os][build_type][apk_type].items()
                ):
                    print(
                        f"│   │   ├── {filename} [hash: {PURPLE}{BOLD}{hash_prefix}{RESET}]"
                    )
    print("..... Done")


print("Directory Tree with Hash Prefixes (first 8 chars):")
print_tree_with_hashes(df)

Directory Tree with Hash Prefixes (first 8 chars):
fedora40/
├── manual/
│   ├── apks-from-device/
│   │   ├── base-arm64_v8a.apk [hash: d296fff8]
│   │   ├── base-master.apk [hash: eb1902b3]
│   │   ├── base-xxhdpi.apk [hash: c65a2691]
│   ├── apks-i-built/
│   │   ├── base-arm64_v8a.apk [hash: a74d526f]
│   │   ├── base-master.apk [hash: 47686026]
│   │   ├── base-xxhdpi.apk [hash: 51119b04]
├── scripted/
│   ├── apks-from-device/
│   │   ├── base-arm64_v8a.apk [hash: d296fff8]
│   │   ├── base-master.apk [hash: eb1902b3]
│   │   ├── base-xxhdpi.apk [hash: c65a2691]
│   ├── apks-i-built/
│   │   ├── base-arm64_v8a.apk [hash: a74d526f]
│   │   ├── base-master.apk [hash: 47686026]
│   │   ├── base-xxhdpi.apk [hash: 51119b04]
fedora41/
├── scripted/
│   ├── apks-from-device/
│   │   ├── base-arm64_v8a.apk [hash: d296fff8]
│   │   ├── base-master.apk [hash: eb1902b3]
│   │   ├── base-xxhdpi.apk [hash: c65a2691]
│   ├── apks-i-built/
│   │   ├── base-arm64_v8a.apk [hash: 4832f3a8]
│   │  

## Build Reproducibility Analysis

Let's analyze the reproducibility of builds by comparing hashes across different environments.

In [4]:
df.groupby(["os", "build_type"])["exec_count"].nunique().reset_index(
    name="execution_times"
).rename_axis(None).set_index(["os", "build_type"])

execution_times
os       build_type                 
fedora40 manual                    1
         scripted                  2
fedora41 scripted                  1
ubuntu24 scripted                  1

> So indeed, 5 total runs, 4 of them scripted as explained in Methodology

In [5]:
df[~df["local_apk"]].groupby("filename")["hash"].nunique().reset_index(
    name="distinct file hashes"
).set_index("filename")

,distinct file hashes
filename,
base-arm64_v8a.apk,1
base-master.apk,1
base-xxhdpi.apk,1


> So, we have only one unique variant of each apk file, which implies that every phone received the same apk bundle (splits) when 

Sanity check: do all the builds in the same environment produce the same apk?

In [6]:
df[df["local_apk"]].groupby(["os", "filename"])["hash"].nunique().reset_index(
    name="distinct file hashes"
).set_index(["os", "filename"])

distinct file hashes
os       filename                                
fedora40 base-arm64_v8a.apk                     1
         base-master.apk                        1
         base-xxhdpi.apk                        1
fedora41 base-arm64_v8a.apk                     1
         base-master.apk                        1
         base-xxhdpi.apk                        1
ubuntu24 base-arm64_v8a.apk                     1
         base-master.apk                        1
         base-xxhdpi.apk                        1

> So, the host operating system does NOT introduce variability with regards to the built of subsequent apk (bunndles)

In [7]:
df[df["local_apk"]].groupby(["filename"])["hash"].nunique()

filename
base-arm64_v8a.apk    2
base-master.apk       3
base-xxhdpi.apk       2
Name: hash, dtype: int64

In [8]:
# Step 1: Group by 'filename' and count unique 'hash' values
unique_hashes_count = df[df["local_apk"]].groupby("filename")["hash"].nunique()

# Step 2: Filter filenames where the unique hash count is greater than 1
diverging_filenames = unique_hashes_count[unique_hashes_count > 1].index

# Step 3: Extract rows where the 'filename' is in the list of filenames with multiple unique hashes
divergent_data = df[df["local_apk"] & df["filename"].isin(diverging_filenames)]

# Step 4: Group by 'filename' and get the relevant columns
result = (
    divergent_data.groupby(["filename", "hash"])[["os", "filepath"]]
    .agg(
        {
            "os": "first",  # Pick the first 'os' for each filename (assuming it's consistent within each group)
            "filepath": "first",  # Pick the first 'filepath' for each filename (assuming it's consistent within each group)
            # 'hash': 'unique'       # Get all unique hash values for each filename
        }
    )
    .reset_index()
)

# Step 5: Show only rows where there are multiple hashes (you can remove this step if needed)
result = result[result["hash"].apply(len) > 1]

# Show the result
result.set_index(["filename", "os"])

hash  \
filename           os                                                            
base-arm64_v8a.apk ubuntu24  4832f3a837a4059db7476c45d995045b0c561388234335...   
                   fedora40  a74d526fd13f4fe3bda7e066c575fa0c931d8da7333b8d...   
base-master.apk    fedora40  4768602685c6e5ceff078093c87e9aba5f19597b0d76c5...   
                   fedora41  dbe76a3649f90f061170c6d3d2071bcddc3c32672bebc2...   
                   ubuntu24  fa1e7eec6dc3e634aae9ccba817d04db46b46c10ca4556...   
base-xxhdpi.apk    ubuntu24  41b12629671ba0778364d671909add3a94380955c5c366...   
                   fedora40  51119b048c0e03525cbb253ed7c19974738b58804908d3...   

                                                                      filepath  
filename           os                                                           
base-arm64_v8a.apk ubuntu24  outputs/ubuntu24-scripted-1/apks-i-built/base-...  
                   fedora40  outputs/fedora40-scripted-1/apks-i-built/base-...  
base-master.apk    fedora40  outputs/fedora40-scripted-1/apks-i-built/base-...  
                   fedora41  outputs/fedora41-scripted-1/apks-i-built/base-...  
                   ubuntu24  outputs/ubuntu24-scripted-1/apks-i-built/base-...  
base-xxhdpi.apk    ubuntu24  outputs/ubuntu24-scripted-1/apks-i-built/base-...  
                   fedora40  outputs/fedora40-scripted-1/apks-i-built/base-...

> What about fedora41?

In [9]:
df[(df["os"] == "fedora41") & (df["local_apk"])][["os", "filename", "filepath", "hash"]]

,os,filename,filepath,hash
27,fedora41,base-arm64_v8a.apk,outputs/fedora41-scripted-1/apks-i-built/base-...,4832f3a837a4059db7476c45d995045b0c561388234335...
28,fedora41,base-xxhdpi.apk,outputs/fedora41-scripted-1/apks-i-built/base-...,41b12629671ba0778364d671909add3a94380955c5c366...
29,fedora41,base-master.apk,outputs/fedora41-scripted-1/apks-i-built/base-...,dbe76a3649f90f061170c6d3d2071bcddc3c32672bebc2...


> So, it produced a similar `base-arm64_v8a.apk` and `base-xxhdpi.apk` as ubuntu24, but different than fedora40. Each os has a unique `base-master.apk`

## Investigation

In [10]:
from build_signal import SignalBuilder

sb = SignalBuilder()

In [11]:
# sb.clone_signal("7.25.2")

In [12]:
from pathlib import Path

sb.setup_apkdiff(Path("./"))


Setting up apkdiff.py...


PosixPath('.')

In [13]:
from apkdiff import ApkDiff

In [14]:
df.columns

Index(['os', 'build_type', 'exec_count', 'apk_type', 'local_apk', 'filename',
       'filepath', 'hash', 'size', 'human_size'],
      dtype='object')

In [19]:
# Group by 'filename' and drop duplicate entries by 'hash'
df_unique = df.groupby("filename").apply(
    lambda group: group.drop_duplicates(subset=["hash"])
)

# Reset the index to get a clean DataFrame
df_unique = df_unique.reset_index(drop=True)

# Display the result
df_unique.set_index(["filename", "os"])

/tmp/ipykernel_13416/3400205333.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_unique = df.groupby('filename').apply(lambda group: group.drop_duplicates(subset=['hash']))


build_type  exec_count          apk_type  \
filename           os                                                  
base-arm64_v8a.apk fedora40   scripted           1  apks-from-device   
                   fedora40   scripted           1      apks-i-built   
                   ubuntu24   scripted           1      apks-i-built   
base-master.apk    fedora40   scripted           1  apks-from-device   
                   fedora40   scripted           1      apks-i-built   
                   ubuntu24   scripted           1      apks-i-built   
                   fedora41   scripted           1      apks-i-built   
base-xxhdpi.apk    fedora40   scripted           1  apks-from-device   
                   fedora40   scripted           1      apks-i-built   
                   ubuntu24   scripted           1      apks-i-built   

                             local_apk  \
filename           os                    
base-arm64_v8a.apk fedora40      False   
                   fedora40       True   
                   ubuntu24       True   
base-master.apk    fedora40      False   
                   fedora40       True   
                   ubuntu24       True   
                   fedora41       True   
base-xxhdpi.apk    fedora40      False   
                   fedora40       True   
                   ubuntu24       True   

                                                                      filepath  \
filename           os                                                            
base-arm64_v8a.apk fedora40  outputs/fedora40-scripted-1/apks-from-device/b...   
                   fedora40  outputs/fedora40-scripted-1/apks-i-built/base-...   
                   ubuntu24  outputs/ubuntu24-scripted-1/apks-i-built/base-...   
base-master.apk    fedora40  outputs/fedora40-scripted-1/apks-from-device/b...   
                   fedora40  outputs/fedora40-scripted-1/apks-i-built/base-...   
                   ubuntu24  outputs/ubuntu24-scripted-1/apks-i-built/base-...   
                   fedora41  outputs/fedora41-scripted-1/apks-i-built/base-...   
base-xxhdpi.apk    fedora40  outputs/fedora40-scripted-1/apks-from-device/b...   
                   fedora40  outputs/fedora40-scripted-1/apks-i-built/base-...   
                   ubuntu24  outputs/ubuntu24-scripted-1/apks-i-built/base-...   

                                                                          hash  \
filename           os                                                            
base-arm64_v8a.apk fedora40  d296fff8691b3632f85725202045875af7e89dafedc9df...   
                   fedora40  a74d526fd13f4fe3bda7e066c575fa0c931d8da7333b8d...   
                   ubuntu24  4832f3a837a4059db7476c45d995045b0c561388234335...   
base-master.apk    fedora40  eb1902b3aa98e15a140e9de574dfa47b78bf8bb85fc556...   
                   fedora40  4768602685c6e5ceff078093c87e9aba5f19597b0d76c5...   
                   ubuntu24  fa1e7eec6dc3e634aae9ccba817d04db46b46c10ca4556...   
                   fedora41  dbe76a3649f90f061170c6d3d2071bcddc3c32672bebc2...   
base-xxhdpi.apk    fedora40  c65a2691cde28650f8934d492bff6c241b2e55eee4d222...   
                   fedora40  51119b048c0e03525cbb253ed7c19974738b58804908d3...   
                   ubuntu24  41b12629671ba0778364d671909add3a94380955c5c366...   

                                 size human_size  
filename           os                             
base-arm64_v8a.apk fedora40  23569519       22MB  
                   fedora40  23561264       22MB  
                   ubuntu24  23552089       22MB  
base-master.apk    fedora40  85037580       81MB  
                   fedora40  85021050       81MB  
                   ubuntu24  84693087       80MB  
                   fedora41  84693087       80MB  
base-xxhdpi.apk    fedora40   1666719        1MB  
                   fedora40   1658464        1MB  
                   ubuntu24   1637689        1MB

In [48]:
# Step 1: Group by the 'filename' index level
grouped = df_unique.groupby("filename")

# Step 2: For each group, generate all pairs of rows (without repetition)
pairs = []
for filename, group in grouped:
    # If the group has more than one row, create combinations of indices
    if len(group) > 1:
        # Generate all unique combinations of row indices for this group
        for index1, index2 in itertools.combinations(group.index, 2):
            pair_data = {
                "filename": filename,
                "pair": (index1, index2),
                "row_1_data": group.loc[index1].to_dict(),
                "row_2_data": group.loc[index2].to_dict(),
            }
            pairs.append(pair_data)

# Step 3: Convert the list of pairs into a DataFrame
pairs_df = pd.DataFrame(pairs)

# Show the pairs DataFrame
pairs_df

,filename,pair,row_1_data,row_2_data
0,base-arm64_v8a.apk,"(0, 1)","{'os': 'fedora40', 'build_type': 'scripted', '...","{'os': 'fedora40', 'build_type': 'scripted', '..."
1,base-arm64_v8a.apk,"(0, 2)","{'os': 'fedora40', 'build_type': 'scripted', '...","{'os': 'ubuntu24', 'build_type': 'scripted', '..."
2,base-arm64_v8a.apk,"(1, 2)","{'os': 'fedora40', 'build_type': 'scripted', '...","{'os': 'ubuntu24', 'build_type': 'scripted', '..."
3,base-master.apk,"(3, 4)","{'os': 'fedora40', 'build_type': 'scripted', '...","{'os': 'fedora40', 'build_type': 'scripted', '..."
4,base-master.apk,"(3, 5)","{'os': 'fedora40', 'build_type': 'scripted', '...","{'os': 'ubuntu24', 'build_type': 'scripted', '..."
5,base-master.apk,"(3, 6)","{'os': 'fedora40', 'build_type': 'scripted', '...","{'os': 'fedora41', 'build_type': 'scripted', '..."
6,base-master.apk,"(4, 5)","{'os': 'fedora40', 'build_type': 'scripted', '...","{'os': 'ubuntu24', 'build_type': 'scripted', '..."
7,base-master.apk,"(4, 6)","{'os': 'fedora40', 'build_type': 'scripted', '...","{'os': 'fedora41', 'build_type': 'scripted', '..."
8,base-master.apk,"(5, 6)","{'os': 'ubuntu24', 'build_type': 'scripted', '...","{'os': 'fedora41', 'build_type': 'scripted', '..."
9,base-xxhdpi.apk,"(7, 8)","{'os': 'fedora40', 'build_type': 'scripted', '...","{'os': 'fedora40', 'build_type': 'scripted', '..."


In [49]:
pairs_df["row_1_data"][0]

{'os': 'fedora40',
 'build_type': 'scripted',
 'exec_count': 1,
 'apk_type': 'apks-from-device',
 'local_apk': False,
 'filename': 'base-arm64_v8a.apk',
 'filepath': 'outputs/fedora40-scripted-1/apks-from-device/base-arm64_v8a.apk',
 'hash': 'd296fff8691b3632f85725202045875af7e89dafedc9df17a455120bff6f1791',
 'size': 23569519,
 'human_size': '22MB'}

In [67]:
# ApkDiff.compare()
for index, test in pairs_df.iterrows():
    r1, r2 = test["row_1_data"], test["row_2_data"]
    print(r1)
    print(r2)
    # ApkDiff().compare(r1['filepath'], r2['filepath'])

    break

{'os': 'fedora40', 'build_type': 'scripted', 'exec_count': 1, 'apk_type': 'apks-from-device', 'local_apk': False, 'filename': 'base-arm64_v8a.apk', 'filepath': 'outputs/fedora40-scripted-1/apks-from-device/base-arm64_v8a.apk', 'hash': 'd296fff8691b3632f85725202045875af7e89dafedc9df17a455120bff6f1791', 'size': 23569519, 'human_size': '22MB'}
{'os': 'fedora40', 'build_type': 'scripted', 'exec_count': 1, 'apk_type': 'apks-i-built', 'local_apk': True, 'filename': 'base-arm64_v8a.apk', 'filepath': 'outputs/fedora40-scripted-1/apks-i-built/base-arm64_v8a.apk', 'hash': 'a74d526fd13f4fe3bda7e066c575fa0c931d8da7333b8de9709472994c453b08', 'size': 23561264, 'human_size': '22MB'}


In [76]:
apkdiff_result_path = pathlib.Path("apkdiff-results")
mismatch_folder = pathlib.Path(f"mismatches")

if apkdiff_result_path.exists() and apkdiff_result_path.is_dir():
    shutil.rmtree(apkdiff_result_path)
apkdiff_result_path.mkdir(parents=True, exist_ok=True)
if mismatch_folder.exists() and mismatch_folder.is_dir():
    shutil.rmtree(mismatch_folder)


for index, row in pairs_df.iterrows():
    r1, r2 = row["row_1_data"], row["row_2_data"]
    l1 = r1["os"] if r1["local_apk"] else "playstore"
    l2 = r2["os"] if r2["local_apk"] else "playstore"
    test_idx_folder = apkdiff_result_path / f"{r1['filename']}_{l1}_vs_{l2}"

    print(f"Test #{index+1}: comparing {r1['filename']} -- {l1} with {l2}")

    res = ApkDiff().compare(r1["filepath"], r2["filepath"])

    if res:
        print("\tAPKS match [OK] ^^")
    else:
        # Step 3: Generate the mismatch folder (simulated command execution)
        diff1_folder = mismatch_folder / "first"
        diff2_folder = mismatch_folder / "second"

        test_idx_folder.mkdir(parents=True, exist_ok=True)
        shutil.move(str(diff1_folder), str(test_idx_folder / "first"))
        shutil.move(str(diff2_folder), str(test_idx_folder / "second"))
        print(f"Output results in {test_idx_folder}")
        # break
    print("=====================")

if mismatch_folder.exists() and mismatch_folder.is_dir():
    shutil.rmtree(mismatch_folder)

Test #1: comparing base-arm64_v8a.apk -- playstore with fedora40
APKs differ on file AndroidManifest.xml! Files extracted to the mismatches/ directory.
Output results in apkdiff-results/base-arm64_v8a.apk_playstore_vs_fedora40
Test #2: comparing base-arm64_v8a.apk -- playstore with ubuntu24
APKs differ on file AndroidManifest.xml! Files extracted to the mismatches/ directory.
Output results in apkdiff-results/base-arm64_v8a.apk_playstore_vs_ubuntu24
Test #3: comparing base-arm64_v8a.apk -- fedora40 with ubuntu24
	APKS match [OK] ^^
Test #4: comparing base-master.apk -- playstore with fedora40
APKs differ on file AndroidManifest.xml! Files extracted to the mismatches/ directory.
APKs differ on file classes2.dex! Files extracted to the mismatches/ directory.
APKs differ on file classes3.dex! Files extracted to the mismatches/ directory.
APKs differ on file classes4.dex! Files extracted to the mismatches/ directory.
APKs differ on file classes5.dex! Files extracted to the mismatches/ dire

TODO: what tf is going on, run apkdiff on the matrix of diverging stuff and check

## Conclusions

Based on the analysis above, we can draw the following conclusions:

1. Build Reproducibility: [This will be filled based on actual results]
2. Size Consistency: [This will be filled based on actual results]
3. Environment Impact: [This will be filled based on actual results]

### Recommendations

Based on these findings, here are some recommendations for improving build reproducibility:

1. [Will be filled based on actual results]
2. [Will be filled based on actual results]
3. [Will be filled based on actual results]